In [1]:
import pylangacq as pla

In [2]:
pla.__version__

'0.11.0'

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
import glob

In [6]:
import os

In [7]:
import pickle

In [8]:
import torch

In [9]:
import nltk

import re
from collections import Counter

import string

import pprint

In [10]:
import math

Data grab

In [11]:
# pd.options.display.max_seq_items = 2000

pd.set_option('display.max_columns', None)

In [12]:
os.chdir('..')

In [13]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [14]:
#ignore header for easier (ordinal) columnar indexing
datasheet = pd.read_excel('CogData_FU_82818.xlsx',header=None)

Row Indices of Memory Responses 

In [15]:
#ignore first (header) column
memory_ix = datasheet.iloc[1:][datasheet.iloc[1:,2].notnull()].index

Record IDs of Memory Respondents [Analyze all scores from each participant]

In [16]:
memory_id = list(datasheet.iloc[memory_ix][0])

Tech respondents

In [17]:
tech_ix = datasheet.iloc[1:][datasheet.iloc[1:,3].notnull()].index

In [18]:
tech_id = list(datasheet.iloc[tech_ix][0])

In [19]:
#each respondent responded to one prompt
len(tech_id) == len(set(tech_id))

True

In [20]:
ix_id_series = pd.Series(tech_id,index=tech_ix)

In [21]:
datasheet.iloc[0][74]

'Total MMSE score'

In [22]:
ix_score_latest_completed = pickle.load(open('ix_score_latest_completed.pkl','rb'))

In [23]:
ix_score_latest_completed = []
for ix in tech_id:
    slice_ix = list(datasheet[datasheet[0]==ix].index)
#     while datasheet.iloc[max(slice_ix)][96] != 'Complete':
    # if there is no mmse score, take most recent score
    while np.isnan(datasheet.iloc[max(slice_ix)][74]):
        slice_ix.remove(max(slice_ix))
    ix_score_latest_completed += [max(slice_ix)]

In [24]:
mem_ix_score_latest_completed = [i for i in ix_score_latest_completed if datasheet.iloc[i,0] in memory_id]

In [25]:
tech_mmse = pd.Series([y for y in datasheet.iloc[ix_score_latest_completed,74]])

In [126]:
# ix by samples 
tech_mmse_dict = {ix:score for (ix,score) in zip(tech_ix,tech_mmse)}

In [127]:
torch.save(tech_mmse_dict,'tech_mmse_dict.pt')

In [26]:
mem_mmse = pd.Series([y for y in datasheet.iloc[mem_ix_score_latest_completed,74]])

In [124]:
mem_mmse_dict = {ix:score for (ix,score) in zip(memory_ix,mem_mmse)}

In [125]:
torch.save(mem_mmse_dict,'mem_mmse_dict.pt')

Load addb vocab glove word embeddings 

In [27]:
import torch

In [28]:
glove_path = './FinalProj/PretrainedWordEmb/'

In [29]:
addbvocab = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [30]:
addbvocab_legacy = pickle.load(open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: './FinalProj/PretrainedWordEmb//addb.vocab.glove.42B.300_words.pkl'

In [31]:
len(addbvocab)

2188

In [52]:
addbvocab['I'] != addbvocab_legacy['I']

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [76]:
for k in addbvocab.keys():

    try:
        if not torch.all(addbvocab[k].eq(addbvocab_legacy[k])):
            print(k)
            print(addbvocab[k])
            print(addbvocab_legacy[k])
    except TypeError as t:
        print(t)
        print(k)
        print(addbvocab[k])
        print(addbvocab_legacy[k])
        continue

eq() received an invalid combination of arguments - got (numpy.ndarray), but expected one of:
 * (Tensor other)
      didn't match because some of the arguments have invalid types: (numpy.ndarray)
 * (Number other)
      didn't match because some of the arguments have invalid types: (numpy.ndarray)

CLITIC
tensor([-6.7711e-03, -2.3817e-03,  2.4845e-02,  3.8487e-02, -1.2772e-01,
         3.3165e-02,  4.8666e-01, -1.6021e-01,  1.6853e-02,  1.6322e-01,
        -1.0224e-01, -5.2618e-02,  3.4685e-02, -1.9040e-02,  3.2686e-02,
         3.5499e-02,  6.3939e-02,  3.8428e-02, -1.1650e-02, -2.5398e-03,
        -1.5485e-02, -2.3090e-02,  1.1053e-02, -2.8602e-02, -2.3924e-02,
         5.4396e-02, -2.7256e-03,  2.9342e-02,  3.3936e-02,  5.7305e-02,
         5.9507e-02,  1.0920e-04, -2.3985e-02,  2.5953e-02,  1.0569e-02,
         7.6946e-03,  1.8665e-02,  1.7798e-02, -2.5355e-02,  2.4376e-02,
         2.5437e-02, -5.5402e-02,  2.5985e-02,  4.1344e-02,  1.8730e-02,
         3.6826e-03, -3.2120e-02,  

In [63]:
addbvocab.keys() == addbvocab_legacy.keys()

True

In [67]:
[v,vl for (v,vl) in zip(addbvocab,addbvocab_legacy) ]

SyntaxError: invalid syntax (<ipython-input-67-dddad3ac77dd>, line 1)

# Helper tokenize

In [32]:
# https://stackoverflow.com/questions/367155/splitting-a-string-into-words-and-punctuation -first comment
# The trick is, not to think about where to split the string, but what to include in the tokens.
# Put any additional punctuation marks you want to use in the right half of the regular expression.
# Anything not explicitely mentioned in the re is silently dropped.

# returns nested sentence

def list_tokenize(document):
#     return re.sub('['+string.punctuation.replace('\'','')+']',' ',response).split()
    sentences = nltk.sent_tokenize(document)
    
    return [nltk.word_tokenize(sent) for sent in sentences]
#     return re.findall(r"[\w']+|[^\s\w]",response)

In [33]:
#information-extraction
#https://www.nltk.org/book/ch07.html
def get_pos(document):
    #sentence segmentation
    sentences = nltk.sent_tokenize(document) 
    #tokenization
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    #pos tagging
    sentences = [nltk.pos_tag(sent) for sent in sentences] 
    
    pos_list = [p for sent in sentences for (t,p) in sent]
    return pos_list
    return sentences

In [129]:
get_pos(datasheet.iloc[memory_ix[3],2])

['PRP$',
 'NN',
 'CC',
 'PRP',
 'VBD',
 'RB',
 'VBN',
 'TO',
 'VB',
 'TO',
 'NNP',
 '.',
 'PRP',
 'VBD',
 'IN',
 'PRP',
 'DT',
 'NN',
 ',',
 'CC',
 'DT',
 'NN',
 ',',
 'PRP',
 'VBD',
 'IN',
 'PRP$',
 'NNS',
 'CC',
 'IN',
 'PRP$',
 'NN',
 ',',
 'CC',
 'VBD',
 ',',
 'JJ',
 'NN',
 '.',
 "''",
 'PRP',
 'VBD',
 'CD',
 'NNS',
 'IN',
 'NNP',
 'IN',
 'PRP$',
 'JJ',
 'NN',
 'TO',
 'NNP',
 ',',
 'CC',
 'PRP',
 'VBD',
 'JJS',
 'IN',
 'DT',
 'NN',
 'JJ',
 'NN',
 ',',
 'PRP',
 'VBD',
 'NNS',
 'IN',
 'NN',
 'IN',
 'IN',
 '$',
 'CD',
 ',',
 'CC',
 'PRP',
 'VBD',
 ',',
 'PRP',
 'VBZ',
 'VB',
 '.',
 "''",
 'PRP',
 'VBD',
 'DT',
 'JJ',
 'NN',
 'IN',
 'DT',
 'NNP',
 ',',
 'VBG',
 'NNP',
 ',',
 'WRB',
 'PRP',
 'MD',
 'VB',
 'IN',
 'DT',
 'NN',
 'CC',
 'VB',
 'DT',
 'JJ',
 'NNS',
 'IN',
 'NN',
 'VBG',
 'DT',
 'NN',
 'CC',
 'DT',
 'NN',
 'IN',
 '.',
 'PRP',
 'VBP',
 'NN',
 'IN',
 'JJ',
 'NN',
 ':',
 'PRP',
 'VBD',
 'IN',
 'DT',
 'NN',
 'IN',
 'DT',
 'NNP',
 'IN',
 'CD',
 'NN',
 'CC',
 'VBD',
 'NNP',
 'NN',


---------------------------------------------

In [128]:
list_tokenize(datasheet.iloc[memory_ix[0],2])

[['When',
  'I',
  'was',
  'a',
  'little',
  'child',
  'I',
  'had',
  'the',
  'same',
  'experience',
  'over',
  'and',
  'over',
  '.'],
 ['It',
  'always',
  'occurred',
  'between',
  'being',
  'awake',
  'and',
  'asleep',
  '.'],
 ['The',
  'experience',
  'had',
  'no',
  'words',
  '...',
  'it',
  'was',
  'all',
  'sensation',
  '.'],
 ['At',
  'a',
  'point',
  'between',
  'my',
  'eyes',
  'I',
  'got',
  'a',
  'feeling',
  'of',
  'all',
  'space',
  'expanding',
  'and',
  'expanding',
  'until',
  'it',
  'became',
  'so',
  'vast',
  'I',
  'was',
  'terrified',
  '.'],
 ['Then',
  'all',
  'space',
  'would',
  'contract',
  'until',
  'it',
  'became',
  'so',
  'tiny',
  'I',
  'would',
  'be',
  'terrified',
  '.'],
 ['I',
  'was',
  'both',
  'afraid',
  'of',
  'having',
  'this',
  'experience',
  'and',
  'welcomed',
  'it',
  '.']]

In [35]:
for sent in get_pos(datasheet.iloc[memory_ix[1],2]):
    print(sent)

WRB
PRP
VBD
RB
JJ
,
VBG
IN
DT
NNP
NN
IN
NNP
,
PRP$
NNS
VBD
PRP$
JJ
NN
CC
PRP
TO
DT
JJ
JJ
VBG
NN
.
PRP
RB
VBD
JJ
:
RB
RB
IN
DT
NN
POS
NN
:
VBN
IN
DT
NNP
TO
VB
NNS
IN
VBG
NN
NNP
NNP
.
DT
NN
VBD
RB
JJ
.
DT
NN
VBD
JJ
,
JJ
VBD
JJ
NN
,
CC
VBD
DT
JJ
JJ
NN
WRB
NNS
MD
VB
.
PRP
VBP
DT
NN
,
DT
JJ
NN
,
DT
NN
IN
DT
NN
,
DT
NN
IN
DT
NN
.
VBG
CC
NN
CC
VBG
VBP
VBN
DT
JJ
NN
IN
PRP$
NN
.


In [183]:
[p for sent in pos_preprocess(datasheet.iloc[memory_ix[1],2]) for (t,p) in sent]

['WRB',
 'PRP',
 'VBD',
 'RB',
 'JJ',
 ',',
 'VBG',
 'IN',
 'DT',
 'NNP',
 'NN',
 'IN',
 'NNP',
 ',',
 'PRP$',
 'NNS',
 'VBD',
 'PRP$',
 'JJ',
 'NN',
 'CC',
 'PRP',
 'TO',
 'DT',
 'JJ',
 'JJ',
 'VBG',
 'NN',
 '.',
 'PRP',
 'RB',
 'VBD',
 'JJ',
 ':',
 'RB',
 'RB',
 'IN',
 'DT',
 'NN',
 'POS',
 'NN',
 ':',
 'VBN',
 'IN',
 'DT',
 'NNP',
 'TO',
 'VB',
 'NNS',
 'IN',
 'VBG',
 'NN',
 'NNP',
 'NNP',
 '.',
 'DT',
 'NN',
 'VBD',
 'RB',
 'JJ',
 '.',
 'DT',
 'NN',
 'VBD',
 'JJ',
 ',',
 'JJ',
 'VBD',
 'JJ',
 'NN',
 ',',
 'CC',
 'VBD',
 'DT',
 'JJ',
 'JJ',
 'NN',
 'WRB',
 'NNS',
 'MD',
 'VB',
 '.',
 'PRP',
 'VBP',
 'DT',
 'NN',
 ',',
 'DT',
 'JJ',
 'NN',
 ',',
 'DT',
 'NN',
 'IN',
 'DT',
 'NN',
 ',',
 'DT',
 'NN',
 'IN',
 'DT',
 'NN',
 '.',
 'VBG',
 'CC',
 'NN',
 'CC',
 'VBG',
 'VBP',
 'VBN',
 'DT',
 'JJ',
 'NN',
 'IN',
 'PRP$',
 'NN',
 '.']

In [99]:
datasheet.iloc[0]

0                                             Record ID
1                                            Event Name
2     Write a short sketch about a memory from your ...
3     How does technology and social media impact th...
4                   Date of Neurobehavioral Status Exam
                            ...                        
92                         Cog State: One Back Accuracy
93                          Ravens Progressive Matrices
94                                Logical Memory Part B
95                         East Boston Immediate Recall
96                                            Complete?
Name: 0, Length: 97, dtype: object

In [ ]:
list_tokenize(datasheet.iloc)

In [36]:
vocabset = set()

In [39]:
list_tokenize(datasheet.iloc[tech_ix[0],3])

[['I',
  'stay',
  'in',
  'touch',
  'with',
  'family',
  'and',
  'friends',
  'through',
  'email',
  '.']]

In [40]:
# for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
for ix in tech_ix:
    for sent in list_tokenize(datasheet.iloc[ix,3]):
        vocabset.update(sent)
#         tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
#         vocabset.update(tokens)

In [41]:
for ix in memory_ix:
    for sent in list_tokenize(datasheet.iloc[ix,2]):
        vocabset.update(sent)

In [42]:
len(vocabset)

2125

db Vocab as gloVe emb 

In [43]:
# sorted(list(addbvocab)) == sorted(list(vocabset))

True

BUILD MOR: POS + GRAMMATICAL CATEGORIES

In [43]:
posset = set()

In [44]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [45]:
addb_posset = torch.load('./FinalProj/pos_dict.pt')

In [163]:
addb_posset

{'': 0,
 'CM': 1,
 'COMP': 2,
 'PRO:EXIST': 3,
 'IN#ADJ': 4,
 'PRE#PART': 5,
 '.': 6,
 'PRE#V': 7,
 'OVER#PART': 8,
 'UN#ADV': 9,
 'PRO:PER': 10,
 'ADV:TEM': 11,
 '+/.': 12,
 'V': 13,
 'PRO:INT': 14,
 'N': 15,
 'PRO:POSS': 16,
 'N:ADJ': 17,
 'DET:DEM': 18,
 'END': 19,
 'PRO:INDEF': 20,
 '+"/.': 21,
 'DET:POSS': 22,
 'OVER#V': 23,
 'DET:ART': 24,
 'INF': 25,
 'OVER#N:GERUND': 26,
 'META': 27,
 'ON': 28,
 'UP#V': 29,
 'MID#N': 30,
 'NEG': 31,
 'MOD': 32,
 '+...': 33,
 'PRO:SUB': 34,
 'UN#PART': 35,
 'PREP': 36,
 'DET:NUM': 37,
 'N:GERUND': 38,
 '+//?': 39,
 'UN#ADJ': 40,
 'COP': 41,
 'N:PT': 42,
 'AUX': 43,
 'ADV': 44,
 'OUT#PART': 45,
 '!': 46,
 'PRO:OBJ': 47,
 'ADJ': 48,
 'N:PROP': 49,
 'MINI#N': 50,
 '+/?': 51,
 '?': 52,
 'POST': 53,
 'N:LET': 54,
 'UN#N': 55,
 'UP#PART': 56,
 'COORD': 57,
 'QN': 58,
 'GRAND#N': 59,
 'CONJ': 60,
 '+..?': 61,
 'PART': 62,
 'BEG': 63,
 'PRO:REL': 64,
 'PRO:REFL': 65,
 'NEO': 66,
 'CO': 67,
 '+".': 68,
 'PRO:DEM': 69,
 '+//.': 70}

In [46]:
datasheet.iloc[0,2]

'Write a short sketch about a memory from your childhood and why it is memorable or important to you. Feel free to discuss an event with family or friends, a place you traveled, or a significant time in your life.  Please limit your response to no more than 1-2 paragraphs.  '

In [47]:
# for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
for ix in tech_ix:
    posset.update(get_pos(datasheet.iloc[ix,3]))
#     for i in range(len(corpus)):
#         pos = set([pos for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
#         posset.update(pos)

In [48]:
for ix in memory_ix:
    posset.update(get_pos(datasheet.iloc[ix,2]))

In [49]:
set(addb_posset.keys())

{'',
 '!',
 '+".',
 '+"/.',
 '+...',
 '+..?',
 '+/.',
 '+//.',
 '+//?',
 '+/?',
 '.',
 '?',
 'ADJ',
 'ADV',
 'ADV:TEM',
 'AUX',
 'BEG',
 'CM',
 'CO',
 'COMP',
 'CONJ',
 'COORD',
 'COP',
 'DET:ART',
 'DET:DEM',
 'DET:NUM',
 'DET:POSS',
 'END',
 'GRAND#N',
 'IN#ADJ',
 'INF',
 'META',
 'MID#N',
 'MINI#N',
 'MOD',
 'N',
 'N:ADJ',
 'N:GERUND',
 'N:LET',
 'N:PROP',
 'N:PT',
 'NEG',
 'NEO',
 'ON',
 'OUT#PART',
 'OVER#N:GERUND',
 'OVER#PART',
 'OVER#V',
 'PART',
 'POST',
 'PRE#PART',
 'PRE#V',
 'PREP',
 'PRO:DEM',
 'PRO:EXIST',
 'PRO:INDEF',
 'PRO:INT',
 'PRO:OBJ',
 'PRO:PER',
 'PRO:POSS',
 'PRO:REFL',
 'PRO:REL',
 'PRO:SUB',
 'QN',
 'UN#ADJ',
 'UN#ADV',
 'UN#N',
 'UN#PART',
 'UP#PART',
 'UP#V',
 'V'}

In [50]:
posset

{'$',
 "''",
 '(',
 ')',
 ',',
 '.',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'TO',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WRB'}

In [51]:
posset.difference(addb_posset)

{'$',
 "''",
 '(',
 ')',
 ',',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'TO',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WRB'}

In [137]:
# rm these tokens
[v for (k,v) in posdict.items() if v not in addb_posset]

['FW', '(', "''", ':', ')', ',', '$']

In [130]:
posset.difference(addb_posset)

{'$',
 "''",
 '(',
 ')',
 ',',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'TO',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WRB'}

In [52]:
#swap with chat term where poss
posdict = {k:k for k in posset}

In [ ]:
# https://talkbank.org/manuals/MOR.html#_Toc17967934
# https://talkbank.org/manuals/MOR.html#_Toc1313898
# nltk.help.upenn_tagset()

In [238]:
posdict['CC'] = 'CONJ'

In [240]:
posdict['CD'] = 'DET:NUM'

In [242]:
posdict['DT'] = 'DET:ART'

In [244]:
posdict['EX'] = 'PRO:EXIST'

In [246]:
posdict['IN'] = 'PREP'

In [248]:
posdict['JJ'] = 'ADJ'

In [250]:
posdict['JJR'] = 'ADJ'

In [251]:
posdict['JJS'] = 'ADJ'

In [253]:
posdict['MD'] = 'MOD'

In [255]:
posdict['NN'] = 'N'

In [227]:
posdict['NNP'] = 'N:PROP'

In [258]:
posdict['NNPS'] = 'N:PROP'

In [259]:
posdict['NNS'] = 'N'

In [261]:
posdict['PDT'] = 'DET:DEM'

In [262]:
posdict['POS'] = 'DET:POSS'

In [267]:
posdict['PRP'] = 'PRO:PER'

In [269]:
posdict['PRP$'] = 'PRO:POSS'

In [271]:
posdict['RB'] = 'ADV'

In [276]:
posdict['RBR'] = 'ADV'

In [277]:
posdict['RBS'] = 'ADV'

In [279]:
posdict['RP'] = 'PART'

In [282]:
posdict['TO'] = 'PREP'

In [283]:
posdict['VB'] = 'V'

In [285]:
posdict['VBD'] = 'V'

In [287]:
posdict['VBG'] = 'PART'

In [288]:
posdict['VBN']  = 'PART'

In [289]:
posdict['VBP'] = 'V'

In [290]:
posdict['VBZ'] = 'V'

In [293]:
sorted(addb_posset)

['',
 '!',
 '+".',
 '+"/.',
 '+...',
 '+..?',
 '+/.',
 '+//.',
 '+//?',
 '+/?',
 '.',
 '?',
 'ADJ',
 'ADV',
 'ADV:TEM',
 'AUX',
 'BEG',
 'CM',
 'CO',
 'COMP',
 'CONJ',
 'COORD',
 'COP',
 'DET:ART',
 'DET:DEM',
 'DET:NUM',
 'DET:POSS',
 'END',
 'GRAND#N',
 'IN#ADJ',
 'INF',
 'META',
 'MID#N',
 'MINI#N',
 'MOD',
 'N',
 'N:ADJ',
 'N:GERUND',
 'N:LET',
 'N:PROP',
 'N:PT',
 'NEG',
 'NEO',
 'ON',
 'OUT#PART',
 'OVER#N:GERUND',
 'OVER#PART',
 'OVER#V',
 'PART',
 'POST',
 'PRE#PART',
 'PRE#V',
 'PREP',
 'PRO:DEM',
 'PRO:EXIST',
 'PRO:INDEF',
 'PRO:INT',
 'PRO:OBJ',
 'PRO:PER',
 'PRO:POSS',
 'PRO:REFL',
 'PRO:REL',
 'PRO:SUB',
 'QN',
 'UN#ADJ',
 'UN#ADV',
 'UN#N',
 'UN#PART',
 'UP#PART',
 'UP#V',
 'V']

In [296]:
# DET:INT not in addb_posseet
posdict['WDT'] = 'DET:POSS'

In [298]:
posdict['WP'] = 'PRO:INT'

In [299]:
'ADV:WH' in addb_posset

False

In [301]:
# ADV:WH not in addb_posset
posdict['WRB'] = 'ADV'

In [303]:
[missing for missing in posset.difference(set(addb_posset.keys())) if posdict[missing] not in set(addb_posset.keys())]

['FW', '(', "''", ':', ')', ',', '$']

In [304]:
torch.save(posdict,'apc_pos_dict.pt')

In [54]:
posdict = torch.load('apc_pos_dict.pt')

In [223]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [222]:
# NN: noun, common, singular or mass
#     common-carrier cabbage knuckle-duster Casino afghan shed thermostat
#     investment slide humour falloff slick wind hyena override subhumanity
#     machinist ...
# NNP: noun, proper, singular
#     Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
#     Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
#     Shannon A.K.C. Meltex Liverpool ...
# NNPS: noun, proper, plural
#     Americans Americas Amharas Amityvilles Amusements Anarcho-Syndicalists
#     Andalusians Andes Andruses Angels Animals Anthony Antilles Antiques
#     Apache Apaches Apocrypha ...
# NNS: noun, common, plural
#     undergraduates scotches bric-a-brac products bodyguards facets coasts
#     divestitures storehouses designs clubs fragrances averages
#     subjectivists apprehensions muses factory-jobs ...

set(addb_posset.keys()).difference(posset)

{'',
 '!',
 '+".',
 '+"/.',
 '+...',
 '+..?',
 '+/.',
 '+//.',
 '+//?',
 '+/?',
 '?',
 'ADJ',
 'ADV',
 'ADV:TEM',
 'AUX',
 'BEG',
 'CM',
 'CO',
 'COMP',
 'CONJ',
 'COORD',
 'COP',
 'DET:ART',
 'DET:DEM',
 'DET:NUM',
 'DET:POSS',
 'END',
 'GRAND#N',
 'IN#ADJ',
 'INF',
 'META',
 'MID#N',
 'MINI#N',
 'MOD',
 'N',
 'N:ADJ',
 'N:GERUND',
 'N:LET',
 'N:PROP',
 'N:PT',
 'NEG',
 'NEO',
 'ON',
 'OUT#PART',
 'OVER#N:GERUND',
 'OVER#PART',
 'OVER#V',
 'PART',
 'POST',
 'PRE#PART',
 'PRE#V',
 'PREP',
 'PRO:DEM',
 'PRO:EXIST',
 'PRO:INDEF',
 'PRO:INT',
 'PRO:OBJ',
 'PRO:PER',
 'PRO:POSS',
 'PRO:REFL',
 'PRO:REL',
 'PRO:SUB',
 'QN',
 'UN#ADJ',
 'UN#ADV',
 'UN#N',
 'UN#PART',
 'UP#PART',
 'UP#V',
 'V'}

In [36]:
# to concat to embedding tensor 
pos_dict = {pos:i for (i,pos) in enumerate(posset)}

In [37]:
pos_dict

{'': 0,
 'CM': 1,
 'COMP': 2,
 'PRO:EXIST': 3,
 'IN#ADJ': 4,
 'PRE#PART': 5,
 '.': 6,
 'PRE#V': 7,
 'OVER#PART': 8,
 'UN#ADV': 9,
 'PRO:PER': 10,
 'ADV:TEM': 11,
 '+/.': 12,
 'V': 13,
 'PRO:INT': 14,
 'N': 15,
 'PRO:POSS': 16,
 'N:ADJ': 17,
 'DET:DEM': 18,
 'END': 19,
 'PRO:INDEF': 20,
 '+"/.': 21,
 'DET:POSS': 22,
 'OVER#V': 23,
 'DET:ART': 24,
 'INF': 25,
 'OVER#N:GERUND': 26,
 'META': 27,
 'ON': 28,
 'UP#V': 29,
 'MID#N': 30,
 'NEG': 31,
 'MOD': 32,
 '+...': 33,
 'PRO:SUB': 34,
 'UN#PART': 35,
 'PREP': 36,
 'DET:NUM': 37,
 'N:GERUND': 38,
 '+//?': 39,
 'UN#ADJ': 40,
 'COP': 41,
 'N:PT': 42,
 'AUX': 43,
 'ADV': 44,
 'OUT#PART': 45,
 '!': 46,
 'PRO:OBJ': 47,
 'ADJ': 48,
 'N:PROP': 49,
 'MINI#N': 50,
 '+/?': 51,
 '?': 52,
 'POST': 53,
 'N:LET': 54,
 'UN#N': 55,
 'UP#PART': 56,
 'COORD': 57,
 'QN': 58,
 'GRAND#N': 59,
 'CONJ': 60,
 '+..?': 61,
 'PART': 62,
 'BEG': 63,
 'PRO:REL': 64,
 'PRO:REFL': 65,
 'NEO': 66,
 'CO': 67,
 '+".': 68,
 'PRO:DEM': 69,
 '+//.': 70}

In [38]:
torch.save(pos_dict,'pos_dict.pt')

In [306]:
len(posset)

39

--------------------------------------------------------------------------------------------------

VOCAB EMB

In [56]:
#https://github.com/Blosc/bcolz
import bcolz

In [57]:
import zipfile

In [58]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [32]:
# with zipfile.ZipFile('./PretrainedWordEmb/glove.42B.300d.zip', 'r') as zip_ref:
#     zip_ref.extractall('./PretrainedWordEmb/')

load pretrained glove (common crawl 42B tokens, 1.9M vocab, uncased, 300d vectors)

In [59]:
glove_path = './FinalProj/PretrainedWordEmb/'

In [52]:
# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         break
    

In [58]:
# words = []
# idx = 0
# word2idx = {}
# vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')

# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)

In [30]:
# vectors = bcolz.carray(vectors[1:].reshape((-1,300)), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')
# vectors.flush()

In [31]:
# pickle.dump(words, open(f'{glove_path}/glove.42B.300_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'{glove_path}/glove.42B.300_idx.pkl', 'wb'))

In [60]:
vectors = bcolz.open(f'{glove_path}/glove.42B.300.dat')[:]

In [61]:
words = pickle.load(open(f'{glove_path}/glove.42B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/glove.42B.300_idx.pkl', 'rb'))

In [62]:
# UNK token -- average of embeddings
average_vec = np.mean(vectors, axis=0)
# print(average_vec)

In [63]:
'UNK' in words

False

In [64]:
type(words)

list

In [65]:
type(word2idx)

dict

In [66]:
words.append('UNK')

In [67]:
vectors.shape

(1917494, 300)

In [68]:
vectors = np.concatenate((vectors,np.reshape(average_vec,(1,300))))

In [69]:
len(vectors)

1917495

In [70]:
word2idx['UNK'] = len(vectors) - 1

In [71]:
len(words)

1917495

In [113]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer

def get_glovembs(sample):
    
    def handle_contractions(x):
        tokenizer = TreebankWordTokenizer()
        x = tokenizer.tokenize(x)
        x = ' '.join(x)
        return x
    
    # preprocess acc to pretrained embeddings 
    # lower
    # replace _, -
    # rm contractions
    # rm nonascii chars
    
    sample = [s for s in sample]
    
    preprocessample = [handle_contractions(t).split()[0] for t in [token.lower().replace('_','-') for token in sample]]
    
    
    preprocessample = [''.join([i if ord(i) < 128 else '' for i in w]) for w in preprocessample]
    
    preprocessample = [t.translate(str.maketrans('', '', string.punctuation)) for t in preprocessample]
    

#     return set([word for word in preprocessample if word not in words])
    # {w: vectors[word2idx[w]] for w in words}
    return {sample[i]: torch.tensor(vectors[word2idx[preprocessample[i]]]) if preprocessample[i] in words else torch.tensor(vectors[word2idx['UNK']]) for i in range(len(sample))}

In [111]:
traceunks = get_glovembs(vocabset)

In [112]:
traceunks

{'',
 '11alpha',
 '2ndteam',
 'devicelogs',
 'dontuse',
 'electroniceven',
 'envirmental',
 'etccwas',
 'gerslyn',
 'intigral',
 'monmarte',
 'precomputer',
 'preinternet',
 'townno',
 'unqua',
 'windowdoors',
 'wlodawski'}

In [95]:
traceunks = set()
for word in vocabset:
    if word not in words:
        traceunks.add(word)

In [97]:
len(traceunks)

302

In [99]:
'liking' in words

True

In [114]:
trace_vocab_embs = get_glovembs(vocabset)

In [115]:
len(trace_vocab_embs)

2125

INADVERTENTLY OVERWROTE addb.vocab.glove.42B.300_words.pkl with trace.* !!!

In [116]:
pickle.dump(trace_vocab_embs, open(f'{glove_path}/trace.vocab.glove.42B.300_words.pkl', 'wb'))

In [255]:
#addb_vocab_embs = get_glovembs(vocabset)

In [257]:
#len(addb_vocab_embs)

2188

In [91]:
trace_vocab_embs['UNK']

KeyError: 'UNK'

In [277]:
# replace CLITIC emb with UNK token (mistaken for word 'clitic')
# mistakenly saved as an np array...overwrote this key with torch.from_numpy(vocab_emb['CLITIC']): saved as addb.vocab.emb.glove.42B.300
addb_vocab_embs['CLITIC'] = vectors[word2idx['UNK']]

In [279]:
pickle.dump(addb_vocab_embs, open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'wb'))

In [280]:
addbvocab = pickle.load(open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'rb'))

In [282]:
len(addbvocab)

2188

In [283]:
list(addbvocab) == list(addb_vocab_embs)

True

In [152]:
len(vocabset_proc)

2188

In [154]:
# list generated from preprocessing steps subsumed into get_glovembs()
len(unks)

26

In [ ]:
# ignore CLITIC

In [183]:
unks

['cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 'doctor-dick',
 'how-about',
 'god-bless-you',
 '',
 'oh-dear',
 'how-come',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']